<a href="https://colab.research.google.com/github/PurviPatel20/Dataset/blob/main/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!pip install -q py2neo

In [77]:
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship
import pandas as pd

In [78]:
graph = Graph("bolt://44.203.75.161:7687",
              user="neo4j",
              password="hoops-formation-ditto")

In [104]:
query1 = """
OPTIONAL MATCH (s:ns0__Ubhadi{rdfs__label:"GJG 22"}) RETURN s.ns0__hasPrice
"""

result1 = graph.run(query1)
for d in result1:
    print(d)

3410


In [80]:

query2 = """
OPTIONAL MATCH (i:ns0__Ubhadi) WHERE i:ns0__Ubhadi RETURN i.rdfs__label As Seed_variety, i.ns0__yearRelease As year, i.ns0__daysMaturity As Days, i.ns0__hasPrice As Price, i.ns0__oilContent As Oil, i.ns0__podAndKernelYield As pod_and_kernel
"""

result2 = graph.run(query2)
for d in result2:
    print(d)

'GJG 22'	2013	'118'	3410	51.6	'1770'
'GJG 31'	2009	'117'	None	48.0	'3483'
'GJG 9'	2009	'103'	3410	49.0	'1663'
'TPG 41'	2004	'122'	None	49.0	'2088'
'GG 8'	2006	'104-107'	None	46.0	'1776'
'GG 7'	2001	'100'	None	49	None
'GG 6'	1999	'115-120'	None	50.3	'2.78'
'GG 5'	1996	101	2310	49.2	'1270'
'TG 37A'	2004	'122'	2800	None	None
'GG 2'	None	'100-105'	2310	None	None
'J 11'	None	'110-115'	None	None	None
'GG 20'	1992	'120'	2310	50.7	'1960'


In [117]:
query3 = """OPTIONAL MATCH (n:ns0__Ubhadi) WHERE n.rdfs__label = 'GG 5' RETURN n.ns0__hasPrice"""

result3 = graph.run(query3)
for d in result3:
    print(d)

2310


In [7]:
!pip install -q neo4j-driver
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.

In [91]:

from neo4j import GraphDatabase
import spacy
import gradio as gr


In [118]:
driver = GraphDatabase.driver("bolt://44.203.75.161:7687",
                              auth=("neo4j",
                                    "hoops-formation-ditto"))

In [93]:
nlp = spacy.load("en_core_web_lg")
nlp.pipe_labels['ner']

['CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

In [119]:

def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None

# que = "what is the price of GG 2?"
# doc = nlp(que)



In [120]:
def chatbot(input, history=[]):
    output = get_answer(input)
    history.append((input, output))
    return history, history

In [85]:
# def get_answer(question):
#     doc = nlp(question)
#     seed_variety = extract_entity(question, ["PRODUCT","ORG"])

#     if seed_variety is None:
#         return "Sorry, I don't understand what seed variety you're asking about."

#     with driver.session() as session:
#         if "price" in question or "priced" in question:
#             result = session.run(f"MATCH (n:ns0_Ubhadi) WHERE n.rdfs_label = '{seed_variety}' RETURN n.ns0_hasPrice")
#             if result.peek() is None:
#                 return f"No information found for the movie '{seed_variety}'"
#             for record in result:
#                 return f"The release date of '{seed_variety}' is: {record['n.ns0_hasPrice']}"
#         else:
#             return "Sorry, I don't understand what you're asking."


In [121]:

def get_answer(question):
    doc = nlp(question)
    seed = extract_entity(question, ["PRODUCT","ORG"])
    # director_name = extract_entity(question, ["PERSON"])
    if seed is None:
        return "Sorry, I don't understand what seed variety you're asking about."

    with driver.session() as session:
        if "price" in question or "priced" in question:
            result = session.run(f"OPTIONAL MATCH (n:ns0__Ubhadi) WHERE n.rdfs__label = '{seed}' RETURN n.ns0__hasPrice")
            if result.peek() is None:
                return f"No information found for the Seed variety '{seed}'"
            for record in result:
                return f"The price '{seed}' is: {record['n.ns0__hasPrice']}"
        else:
            return "Sorry, I don't understand what you're asking."

In [42]:
# nlp = spacy.load("en_core_web_lg")
# nlp
# doc = nlp("What is the price of GG 7?")
# doc

In [122]:

gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e5685287fbad12837a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e5685287fbad12837a.gradio.live


In [43]:
# doc.ents.